In [2]:
#IMPORT MODULES
import pandas as pd
import numpy as np
import time
# SENTIMENT ANALYSIS USING VADER
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, precision_score, recall_score, classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [85]:
len(df)

32615

In [20]:
len(df1)

48225

In [18]:
columns = [
    "MainBranch",
    "Age",
    "RemoteWork",
    "EdLevel",
    "YearsCode",
    "YearsCodePro",
    "DevType",
    "OrgSize",
    "PurchaseInfluence",
    "TechList",
    "Country",
    "Currency",
    "CompTotal",
    "AISelect",
    "AISent",
    "AIBen",
    "WorkExp",
    "Industry"
]

df1 = df1[columns]

In [19]:
df1 = df1[df1["CompTotal"].notnull()]

In [12]:
df1 = df1.dropna()

In [14]:
df1 = pd.read_csv("Datasets/survey_results_public.csv")
df = pd.read_csv("Updated Dataset.csv")

In [ ]:
df1['LanguageHaveWorkedWith']

In [48]:
def shorten_categories(categories, cutoff):
    categorical_map = {}
    for i in range(len(categories)):
        if categories.values[i] >= cutoff:
            categorical_map[categories.index[i]] = categories.index[i]
        else:
            categorical_map[categories.index[i]] = 'Other'
    return categorical_map

In [49]:
country_map = shorten_categories(df.Country.value_counts(), 400)
df['Country'] = df['Country'].map(country_map)
df.Country.value_counts()

Country
Other    9626
177      8005
59       2682
175      2305
72       1524
30       1422
56       1074
131       918
23        915
115       837
8         768
154       764
79        712
159       632
160       431
Name: count, dtype: int64

In [50]:
df = df[df["Salary"] <= 250000]
df = df[df["Salary"] >= 10000]
df = df[df['Country'] != 'Other']

In [ ]:
df

In [57]:
df['Age'].value_counts()

Age
1    8782
2    5850
3    2031
0    1777
4     723
5     108
6      10
7       6
Name: count, dtype: int64

In [51]:
X = df[[
    "Age",
    "RemoteWork",
    "EdLevel",
    "YearsCode",
    "Country",
    "WorkExp",
    "Industry"
]]

In [52]:
Y = df['Salary']

In [53]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42)

In [54]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [55]:
classifiers = {}

# Linear Regression
classifiers["LinearRegression"] = LinearRegression()

# # Decision Tree
classifiers["DecisionTree"] = DecisionTreeRegressor()

# # Random Forest
classifiers["RandomForest"] = RandomForestRegressor()

# # XGBoost
classifiers["XGBoost"] = XGBRegressor()

# # LightGBM
# classifiers["LightGBM"] = LGBMRegressor()

# # CatBoost
# classifiers["CatBoost"] = CatBoostRegressor(silent=True)

In [56]:
# Create an empty DataFrame to store the results
columns = ['Model', 'Run Time (minutes)', 'MAE', 'MSE', 'RMSE', 'R2']
df_models = pd.DataFrame(columns=columns)

# Loop through your regression models
for key, clf in classifiers.items():
    # STARTING TIME
    start_time = time.time()
    # TRAIN CLASSIFIER ON TRAINING DATA
    clf.fit(X_train_scaled, y_train)
    # MAKE PREDICTIONS USING CURRENT CLASSIFIER
    predictions = clf.predict(X_test_scaled)
    # CALCULATE REGRESSION METRICS
    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    rmse = mean_squared_error(y_test, predictions, squared=False)  # Calculate RMSE
    r2 = r2_score(y_test, predictions)

    row = {'Model': key,
           'Run Time (minutes)': round((time.time() - start_time) / 60, 2),
           'MAE': mae,
           'MSE': mse,
           'RMSE': rmse,
           'R2': r2
           }

    df_models = pd.concat([df_models, pd.DataFrame([row])], ignore_index=True)

# Sort the DataFrame by R-squared (R2) in descending order
df_models = df_models.sort_values(by='R2', ascending=False)

# PRINT THE MODELS WITH REGRESSION METRICS [SORTED]
print(df_models)

              Model  Run Time (minutes)           MAE           MSE  \
3           XGBoost                0.00  28957.987364  1.552640e+09   
2      RandomForest                0.04  31211.171885  1.789830e+09   
0  LinearRegression                0.00  41958.498763  2.719233e+09   
1      DecisionTree                0.00  39943.514844  3.016460e+09   

           RMSE        R2  
3  39403.548031  0.513742  
2  42306.384951  0.439459  
0  52146.264453  0.148387  
1  54922.306023  0.055301  


Country
Other    9626
177      8005
59       2682
175      2305
72       1524
30       1422
56       1074
131       918
23        915
115       837
8         768
154       764
79        712
159       632
160       431
Name: count, dtype: int64

In [12]:
df['RemoteWork'].value_counts()

RemoteWork
2    14199
0    13863
1     4527
3       26
Name: count, dtype: int64

In [62]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(X, Y)
feature_importances = model.feature_importances_

In [63]:
feature_importances

array([0.00510363, 0.01656425, 0.02091029, 0.03272545, 0.06232747,
       0.13599447, 0.05917951, 0.05139641, 0.01657213, 0.01501617,
       0.02633969, 0.40580779, 0.01341024, 0.02066656, 0.02446022,
       0.04819797, 0.04532776])

In [61]:
X = df[[
    "MainBranch",
    "Age",
    "RemoteWork",
    "EdLevel",
    "YearsCode",
    "YearsCodePro",
    "DevType",
    "OrgSize",
    "PurchaseInfluence",
    "TechList",
    "Country",
    "Currency",
    "AISelect",
    "AISent",
    "AIBen",
    "WorkExp",
    "Industry"
]]

In [64]:
map_ = {}

for i, a in enumerate(X):
    map_[a] = feature_importances[i]

In [66]:
sorted(map_.items(), key = lambda x: x[1])

[('MainBranch', 0.005103631115356308),
 ('AISelect', 0.013410240175554443),
 ('TechList', 0.015016169020112385),
 ('Age', 0.016564246534603047),
 ('PurchaseInfluence', 0.016572132348836227),
 ('AISent', 0.020666557539747773),
 ('RemoteWork', 0.02091029195603425),
 ('AIBen', 0.024460219504638916),
 ('Country', 0.026339691112407887),
 ('EdLevel', 0.0327254466803377),
 ('Industry', 0.045327762475215326),
 ('WorkExp', 0.0481979699372621),
 ('OrgSize', 0.051396406823339374),
 ('DevType', 0.05917950782351618),
 ('YearsCode', 0.062327472568785416),
 ('YearsCodePro', 0.1359944680059651),
 ('Currency', 0.40580778637828757)]

In [67]:
df['Age_Truncate'] = df['Age']

In [68]:
df['Age_Truncate'] = df['Age_Truncate'].replace([7, 6, 5], 5)

In [69]:
df['Age_Truncate'].value_counts()

Age_Truncate
1    8782
2    5850
3    2031
0    1777
4     723
5     124
Name: count, dtype: int64

In [70]:
df['MainBranch'].value_counts()

MainBranch
0    18102
2     1185
Name: count, dtype: int64

In [71]:
df['AISelect'].value_counts()

AISelect
2    7821
0    6656
1    4810
Name: count, dtype: int64

In [73]:
df1 = df[df["Country"] == "India"]

In [74]:
df1["Country"].value_counts()

Series([], Name: count, dtype: int64)

In [78]:
df['Country'].value_counts()

Country
177    7160
59     2634
175    2279
30     1333
56     1061
115     822
154     756
8       722
23      715
79      703
131     550
160     417
72      109
159      26
Name: count, dtype: int64

In [17]:
df["Currency"].value_counts()

Currency
131    9148
41     8770
44     2325
57     1498
24     1414
       ... 
85        1
89        1
73        1
25        1
11        1
Name: count, Length: 122, dtype: int64